In [1]:
import cv2
import os
import numpy as np

In [2]:
def emotionImage(emotion):
	# Emojis
	if emotion == 0: 		image = cv2.imread('./Emojis/Enojado.png', cv2.IMREAD_UNCHANGED)
	if emotion == 1: 	image = cv2.imread('./Emojis/Felicidad.png', cv2.IMREAD_UNCHANGED)
	if emotion == 2: 	image = cv2.imread('./Emojis/Neutro.png', cv2.IMREAD_UNCHANGED)
	if emotion == 3: 	image = cv2.imread('./Emojis/Sorpresa.png', cv2.IMREAD_UNCHANGED)
	if emotion == 4: 	image = cv2.imread('./Emojis/Tristeza.png', cv2.IMREAD_UNCHANGED)
	return image

In [3]:
emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()
emotion_recognizer.read('./Modelos/modeloLBPH.xml')

In [4]:
dataPath = './fotos_entrenamiento/'
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)

imagePaths= ['Enojo', 'Felicidad', 'Neutro', 'Sorpresa', 'Tristeza']


In [5]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else: print('Camera 0')
    else: print('Camera 1')
else: print('Camera 0')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

while True:

	ret,frame = cap.read()
	if ret == False: break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	auxFrame = gray.copy()
 
	faces = faceClassif.detectMultiScale(gray,
									scaleFactor=1.1,
									minNeighbors=3,
									minSize=(5, 5),
								)

	for (x,y,w,h) in faces:
		color = (0, 0, 0)
  
		# Detección del rostro
		rostro = auxFrame[y:y+h,x:x+w]
		rostro = cv2.resize(rostro,(150,150), cv2.INTER_CUBIC)
		result = emotion_recognizer.predict(rostro)
  
		# Color de la palabra y la caja en función de la emoción
		if result[1] < 60:
			#print(result)
			if(result[0] == 0): color = (50, 50, 255)		# Rojizo
			elif(result[0] == 1): color = (50, 255, 100)	# Verde
			elif(result[0] == 2): color = (255, 255, 255)	# Blanco
			elif(result[0] == 3): color = (255, 255, 75)	# Celeste
			elif(result[0] == 4): color = (255, 50, 75)		# Azul
   
			# Selección del emoticono en función del sentimiento
			image = emotionImage(result[0])

			
			# Valores auxiliares
			resized = cv2.resize(image, (w, h), interpolation = cv2.INTER_AREA)
			filas = resized.shape[0]
			col = w
			alto = filas // 3
   
			#Emoticono
			n_frame = frame[y - h + alto:y + alto, x:x + col]
			image = resized[:, :, 3]
			image_inv = cv2.bitwise_not(image)
                
			bg_black = cv2.bitwise_and(resized, resized, mask = image)
			bg_black = bg_black[:, :, 0:3]
			bg_frame = cv2.bitwise_and(n_frame, n_frame, mask = image_inv)
                    
			result_im = cv2.add(bg_black, bg_frame)
			frame[y-filas+alto:y+alto, x:x+col] = result_im
   
			
			# Texto de la emoción
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x+50,y+200),2,1.1,(color),1,cv2.LINE_AA)
			
   			#Caja que contiene la cara
   			#cv2.rectangle(frame, (x,y),(x+w,y+h),color,2)
			#cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-10),2,1.1,(color),1,cv2.LINE_AA)
			#cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
   
		else:
			cv2.putText(frame,'No identificado',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			#cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

	cv2.imshow('Vigila tus sentimientos',frame)
	k = cv2.waitKey(1)
	if k == 27:
		break

cap.release()
cv2.destroyAllWindows()

Camera 0
